# Standalone Python Cron Job to grant Pro licenses to new users

In [ ]:
# This is the operation we will run in a cron job for new users
from arcgis.gis import GIS
import requests
import time
import csv
import json
import pandas
from time import strftime

# Make the connection to VT ArcGIS Online
with open('vtActiveConfig.json') as configFile:
    myConfig = json.load(configFile)

# now connect
gis = GIS(myConfig['agolOrg']['url'],username=myConfig['agolOrg']['username'],password=myConfig['agolOrg']['password'])    


# Get the New User role
# TODO: Use REST lookup to do this a bit more elegantly
new_user_role = 'pH1lPndPVtVbrE6l'


# Retrieve all the entitlements in the org
lics = gis.admin.license.all()
licensedProducts = []   # array of the specific Products for which there may be entitlements
licenseArr = []         # array of the actual licenses for the Products
for lic in lics:
    try:
        licensedProducts.append(json.loads(str(lic.properties))['listing']['title'])
    except KeyError:
        pass

entDf = pandas.DataFrame()  # this stores your organization's entitlements total/assigned/remaining for each product in a dataframe
for product in licensedProducts:
    try:
        theLicense = gis.admin.license.get(product)
        licenseArr.append(theLicense)
        thisProductReportDf = theLicense.report #gets the total/assigned/remaining licenses for each product
        thisProductReportDf['Product'] = product
        entDf = entDf.append(thisProductReportDf)
    except KeyError:
        pass
    
# Create a dictionary of the form used by the UMich license assignment script
# See also: https://gist.github.com/sspeery/190d6dc45bc8f4c4b3ba0e2705850463
entitlements = dict.fromkeys(entDf['Product'].unique())
for key in entitlements:
    entitlements[key]=[]
for index, row in entDf.iterrows():
    entitlements[row['Product']].append(row['Entitlement'])

# Retrieve the actual licenses for later use
try:
    licenses = {lic: gis.admin.license.get(lic) for lic in entitlements}
    #print('License information retrieved.\n')
except Exception as ex:
    print('Unable to retrieve license information')
    print(ex)

# Find new users
try:
    new_users = gis.users.search(new_user_role, max_users=9999)
    if len(new_users) == 0:
        print('No new users found.')
    else:
        print(str(len(new_users))+" new users found.")
except Exception as ex:
    print('Unable to perform user search.')
    print(ex)
    print('Exiting.')

# Grant all licenses to all users
for u in new_users:
    print('Updating user: {0}'.format(u.username))
    try:
        for license_type in entitlements:
            lic = licenses[license_type]
            lic.assign(username=u.username, entitlements=entitlements[license_type])
            print('{0} entitlements granted to user.'.format(license_type) + u.username)
    except Exception as ex:
        print('Error encountered while assigning license {0} to user.'.format(license_type) )
        print(ex)
        print('Exiting.')

# Enable ESRI access for new user by changing userType from 'arcgisonly' to 'both'
    try:
        u.esri_access = True  # Note: This only works with arcgis package v. 1.3 or later
        print('ESRI access enabled for user. '+ u.username)
    except Exception as ex:
        print('Error enabling ESRI access for user.')
        print(ex)
        print('Exiting.')

# Change new users' role from "Publisher (New User)" to the standard "Publisher".
    try:
        u.update_role('org_publisher')
        print('User role updated.'+  u.username+'\n')
    except Exception as ex:
        print('Error changing user role')
        print(ex)
        print('Exiting.')
        

# Add this new user to a Google Group for sending system notifications.
# This part is under construction.
#    try:
#        # TODO: Add user to a Google group.
#        # API for this "might" be https://developers.google.com/admin-sdk/directory/
#    except Exception as ex:
#        print('Error adding user to listserv')
#        print(ex)
#        print('Exiting.')

